<a href="https://colab.research.google.com/github/ymoslem/Adaptive-MT-LLM/blob/main/Semantic_Search_Faiss_Multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual Semantic Search with Faiss

This notebook demonstrates how to use Faiss and Sentence-Transformers to perform multilingual semantic search. We have a query in English ("education and schools in Germany") and we retrieve top results from documents in the Ukrainian language.

* Faiss: https://github.com/facebookresearch/faiss
* Faiss tutorial: https://www.pinecone.io/learn/series/faiss/faiss-tutorial/
* Sentence-Transformers: https://github.com/UKPLab/sentence-transformers
* Sentence-Transformers documentation: https://sbert.net/

* Notebooks for some other similarity techniques [here](https://github.com/ymoslem/Sentence-Similarity).

In [1]:
!pip3 install faiss-gpu sentence_transformers -q

In [1]:
# Download files
!git clone https://github.com/ymoslem/Notion-Scraper.git -q

%cd Notion-Scraper/output/
!ls

/content/Notion-Scraper/output
Education	     Ірландія.json  Нідерланди.json  Угорщина.json
Австрія.json	     Ісландія.json  Німеччина.json   Україна.json
Аргентина.json	     Іспанія.json   Норвегія.json    Фінляндія.json
Бельгія.json	     Італія.json    Польща.json      Франція.json
Болгарія.json	     Канада.json    Португалія.json  Хорватія.json
Великобританія.json  Кіпр.json	    Румунія.json     Чехія.json
Греція.json	     Країна.json    Сербія.json      Чорногорія.json
Грузія.json	     Латвія.json    Словаччина.json  Швейцарія.json
Данія.json	     Литва.json     Словенія.json    Швеція.json
Естонія.json	     Мексика.json   США.json
Ізраїль.json	     Молдова.json   Туреччина.json


In [3]:
import json
import os

work_dir = "."
json_files = [file_name for file_name in os.listdir(work_dir) if file_name.endswith(".json")]

corpus = []

for json_file in json_files:
    with open(os.path.join(work_dir,json_file)) as json_input:
        json_content = json.load(json_input)
        for item in json_content:
            url = item["url"]
            text_paragraphs = item["text"].split("\n")
            text_paragraphs = [(para, json_file[:-5], item["topic"], item["url"]) for para in text_paragraphs if len(para.split())>10 \
                               and (para, json_file[:-5], item["topic"], item["url"]) not in corpus]
            corpus += text_paragraphs

corpus[0:5]

[('Виїзд громадян України за кордон можливий за будь-якого наявного документа, що підтверджує особу, включно з "внутрішнім" паспортом України, ID-карткою або документами в Дії.',
  'Нідерланди',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/8b7a1bdcb55a45409662e4f2dd9ebfe3'),
 ('Зауважте! Виїхати до сусідніх країн ЄС без біометричного закордонного паспорта можна лише один раз.',
  'Нідерланди',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/8b7a1bdcb55a45409662e4f2dd9ebfe3'),
 ('В період дії воєнного стану виїзд дітей до 16 років можливий у супроводі одного з батьків без нотаріально посвідченої згоди другого з батьків, а також у супроводі баби, діда, повнолітніх брата або сестри, мачухи, вітчима або інших осіб, уповноважених одним з батьків письмовою заявою, завіреною органом опіки. Дитині потрібно мати закордонний паспорт або свідоцтво про народження (за відсутності паспорта).',
  'Нідерланди',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/8b7a1bd

In [4]:
from sentence_transformers import SentenceTransformer

corpus_sentences = [item[0] for item in corpus]

embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

corpus_embeddings = embedder.encode(corpus_sentences, convert_to_numpy=True)

In [5]:
# Save corpus_embeddings to a file to be able to load later
import pickle

with open("corpus_embeddings_uk.pkl", "wb") as embeddings:
    pickle.dump({'corpus': corpus, 'embeddings': corpus_embeddings}, embeddings)

In [6]:
# To load the embeddings later from the file instead of creating from scratch
import pickle

with open("corpus_embeddings_uk.pkl", "rb") as embeddings:
    data = pickle.load(embeddings)
    corpus = data['corpus']
    corpus_sentences = [item[0] for item in corpus]
    corpus_embeddings = data['embeddings']

In [7]:
corpus_embeddings.shape

(8238, 384)

In [8]:
import faiss

embedding_size = 384
n_clusters = 128
top_k_hits = 20

quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters)

# Number of clusters to explorer at search time.
# We will search for nearest neighbors in 10 clusters
index.nprobe = 8

### Create the FAISS index
print("Start creating FAISS index")

# Train the index to find a suitable clustering
index.train(corpus_embeddings)

# Add all embeddings to the index
index.add(corpus_embeddings)

print("Number of embeddings indexed:", index.ntotal)

Start creating FAISS index
Number of embeddings indexed: 8238


In [42]:
from sentence_transformers import SentenceTransformer

queries = ["education and schools in Germany"]
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

query_embeddings = model.encode(queries)

# Search in FAISS. It returns a matrix with distances and corpus ids.
distances, corpus_ids = index.search(query_embeddings, k=top_k_hits)

print(corpus_ids, "\n")

results = sorted([result for result in zip(distances.flatten(), corpus_ids.flatten())])
print(results, "\n")

for distance, idx in results:
    print(corpus_sentences[idx])
    print(f"Read more: {corpus[idx][1]} - {corpus[idx][2]}: {corpus[idx][3]}")
    print(f"Distance: {round(distance.item(), 2)}\n")

[[2772 2771 5476 5478 2903 2774 1951 2773 5958 2878  214 2077 1947 2131
   506 2876 7040 1948 5487 2987]] 

[(9.90493, 2772), (12.193857, 2771), (17.366167, 5476), (17.789064, 5478), (18.18654, 2903), (18.472946, 2774), (18.633497, 1951), (18.761229, 2773), (18.841148, 5958), (19.09331, 2878), (19.265436, 214), (19.470036, 2077), (19.66442, 1947), (19.77827, 2131), (19.842066, 506), (19.857483, 2876), (19.93596, 7040), (20.255209, 1948), (20.553595, 5487), (20.602602, 2987)] 

Як влаштувати дитину до школи в Німеччині? | Розповідь інсайдера |Відповіді на всі запитання |
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686
Distance: 9.9

Після оформлення та отримання відповідних документів, виникнуть питання в яку школу віддавати дітей і яким чином тут це налаштовано. Більше інформації про систему освіти у Німеччині можна прочитати тут.
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686
Distance: 12.19

За

In [16]:
results[:5]

[(9.90493, 2772),
 (12.193857, 2771),
 (17.366167, 5476),
 (17.789064, 5478),
 (18.18654, 2903)]

In [15]:
# Reranking input [(query, paragraph), (query, paragraph), (query, paragraph), ...]

reranker_input = [(queries[0], corpus[result[1]][0]) for result in results]
reranker_input[:5]

[('education and schools in Germany',
  'Як влаштувати дитину до школи в Німеччині? | Розповідь інсайдера |Відповіді на всі запитання |'),
 ('education and schools in Germany',
  'Після оформлення та отримання відповідних документів, виникнуть питання в яку школу віддавати дітей і яким чином тут це налаштовано. Більше інформації про систему освіти у Німеччині можна прочитати тут.'),
 ('education and schools in Germany',
  'Зарахування до ясла/дошкільного закладу здійснюється через муніципалітет, у якому ви проживаєте у Швеції. Оскільки процес такий самий, як і для школи, те саме стосується. Прикладом у Стокгольмі є Nacka, а інформацію про школу можна знайти тут: https://www.nacka.se/forskola-skola/ukraina-information-forskola-skola/faq-ukraina-forskola-skola/. Зазвичай сайт шведською мовою, але є можливість перекласти його у верхній частині сторінки. Ви повинні надати інформацію про причину вашого перебування в Швеції, особисту інформацію, таку як ім’я, дата народження, адреса, чи є ви

In [12]:
# [Optional] Reranking
# After retrieving the top-k candidates, we can re-rank them with a cross-encoder model

from sentence_transformers import CrossEncoder

model = CrossEncoder("amberoad/bert-multilingual-passage-reranking-msmarco", max_length=512)
reranker_scores = model.predict(reranker_input)

reranker_scores

array([[ 3.5964847 , -2.734025  ],
       [ 0.9109005 , -0.46546054],
       [ 5.3045406 , -4.2468195 ],
       [ 5.399157  , -4.3299828 ],
       [ 5.1483917 , -4.1939054 ],
       [ 2.9685948 , -2.0185874 ],
       [ 4.5973735 , -3.622678  ],
       [ 4.995662  , -4.0253553 ],
       [ 5.5680523 , -4.724469  ],
       [ 2.6057577 , -1.7372148 ],
       [ 4.561674  , -3.5567884 ],
       [ 5.113559  , -4.169189  ],
       [ 4.5246477 , -3.4602773 ],
       [ 5.7684865 , -4.8660135 ],
       [ 5.731863  , -4.7733545 ],
       [ 4.248139  , -3.2506905 ],
       [ 3.7729554 , -2.8010938 ],
       [ 5.516426  , -4.507867  ],
       [ 5.321773  , -4.2480187 ],
       [ 5.211706  , -4.23989   ]], dtype=float32)

In [36]:
# full hits from the corpus with links
full_hits = [[result[0], corpus[result[1]]] for result in results]
reranker_output = zip(reranker_scores, full_hits)

# Compare the results before and after reranking
# for score, hit in zip(reranker_scores, full_hits):
#   print(score, hit)


sorted_reranked_output = sorted([(score[0], hit[1]) for score, hit in reranker_output])

for score, hit in sorted_reranked_output:
  print(f"{hit[0]} \nRead more: {hit[1]} - {hit[2]}: {hit[3]} \nDistance: {round(score.item(), 2)}\n")

Після оформлення та отримання відповідних документів, виникнуть питання в яку школу віддавати дітей і яким чином тут це налаштовано. Більше інформації про систему освіти у Німеччині можна прочитати тут. 
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686 
Distance: 0.91

Університет Людвіга Максиміліян (ЛМУ) в Мюнхені збирає пожертви на підтримку студентів та професорів різними способами (разова фінансова підтримка / стипендії / допомога для боротьби з німецькою владою / консультуванням щодо німецьких класів та варіантів навчання в ЛМУ). 
Read more: Німеччина - Освіта та мовні курси: https://uahelpinfo.notion.site/d6214a719110464d88cb2609515d7a1e 
Distance: 2.61

Усі діти, які проживають у Німеччині, включаючи дітей, які рятуються від війни в Україні, мають право на освіту. Кожен з 16 федеральних держав (Бундеслайндер) має свої правила, але всі діти, включаючи біженців, зобов'язані відвідувати школу, починаючи з 6 років. 
Read more: Німеччина -

# GPU

In [14]:
# To load the embeddings later from the file instead of creating from scratch
import pickle

with open("corpus_embeddings_uk.pkl", "rb") as embeddings:
    data = pickle.load(embeddings)
    corpus = data['corpus']
    corpus_sentences = [item[0] for item in corpus]
    corpus_embeddings = data['embeddings']

In [15]:
import os

# Which GPU to use (if you have multiple GPUs)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # or "0,1" for multiple gpus

# For debugging CUDA errors
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [16]:
!echo $CUDA_VISIBLE_DEVICES

0


In [17]:
# Single GPU

import faiss

embedding_size = 384
n_clusters = 16
top_k_hits = 10

quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters, faiss.METRIC_L2)

# Number of clusters to explorer at search time.
# We will search for nearest neighbors in 8 clusters
index.nprobe = 8

ngpus = faiss.get_num_gpus()
print("Number of GPUs:", ngpus)

res = faiss.StandardGpuResources()  # use a single GPU
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index)
print("Now the index in on GPU.")

# Train the index to find a suitable clustering
assert not gpu_index_flat.is_trained
gpu_index_flat.train(corpus_embeddings)
assert gpu_index_flat.is_trained
print("Training complete!")

gpu_index_flat.add(corpus_embeddings)  # add vectors to the index
print(gpu_index_flat.ntotal, "added.")

Number of GPUs: 1
Now the index in on GPU.
Training complete!
8238 added.


In [18]:
# Multiple GPUs

import faiss

embedding_size = 384
n_clusters = 64
top_k_hits = 10


quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters)

# Number of clusters to explorer at search time.
# We will search for nearest neighbors in 8 clusters
index.nprobe = 8


print("Moving index to gpu before training")

ngpus = faiss.get_num_gpus()
print("Number of GPUs:", ngpus)

gpu_index_flat = faiss.index_cpu_to_all_gpus(index)
print("Now the index in on GPU.")

# Train the index to find a suitable clustering
assert not gpu_index_flat.is_trained
gpu_index_flat.train(corpus_embeddings)
assert gpu_index_flat.is_trained
print("Training complete!")

# Add vectors to the index
gpu_index_flat.add(corpus_embeddings)
print(gpu_index_flat.ntotal, "added.")

Moving index to gpu before training
Number of GPUs: 1
Now the index in on GPU.
Training complete!
8238 added.
